In [ ]:
import heapq
import scipy.stats as sts
import numpy as np
import random

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')



from functools import reduce

## Feedback 

I want to gain feedback on the implementation of OOP. I build the model from first principles, articulating how and why things work step by step, explaining abstractions, but also placing a due amount of effort into unit testing. There are so many possible states of the system, we need to make sure what we know conceptually is well-encoded. 


In [ ]:
# A schedule contains events, but the scheduling itself is implemented through the logic of the events. 
# We have one main event schedule.

In [ ]:
class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''

    def __init__(self, timestamp, function, *args, **kwargs) -> None:
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        return self.timestamp < other.timestamp

    def run(self, schedule):
        self.function(schedule, *self.args, **self.kwargs)

class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    '''
    
    def __init__(self):
        self.now = 0
        self.queue = []

    def add_event_at(self, timestamp, function):
        heapq.heappush(self.queue, Event(timestamp, function, *args, **kwargs))

    def add_event_after(self, interval, function, *args, **kwargs):
        heapq.heappush(
            self.queue, 
            Event(self.now + interval, function, *args, **kwargs)
        )

    def next_event_time(self):
        #Min heaped so first entry is minimum of timestamp = earliest
        return self.queue[0].timestamp

    def run_next_event(self):
        next_event = heapq.heappop(self.queue)

        #update time
        self.now = next_event.timestamp

        next_event.run(self)

    def print_events(self):
        print(repr(self))

        for event in sorted(self.queue):
            print(f'   {event.timestamp}: {event.function.__name__}')

class Queue:
    def __init__(self, service_rate) -> None:
        self.service_time  = 1 / service_rate
        self.people_in_queue = 0
        self.people_being_served = 0 
        
    def add_customer(self, schedule):
        #add a customer to the schedule
        self.people_in_queue += 1

        if self.people_being_served < 1:
            schedule.add_event_after(0, self.start_serving_customer)

    def start_serving_customer(self, schedule):
        self.people_in_queue -= 1
        self.people_being_served += 1

        schedule.add_event_after(
            self.service_time,
            self.finish_serving_customer
        )

    def finish_serving_customer(self, schedule):
        self.people_being_served -= 1
        if self.people_in_queue > 0:
            schedule.add_event_after(0, self.start_serving_customer)

class BusSystem:
    def __init__(self, arrival_rate, service_rate):
        self.queue = Queue(service_rate)
        self.arrival_distribution = sts.expon(scale = 1/arrival_rate)

    def add_customer(self, schedule):
        self.queue.add_customer(schedule)
        schedule.add_event_after(self.arrival_distribution.rvs(), self.add_customer)
    
    def run(self, schedule):
        schedule.add_event_after( self.arrival_distribution.rvs(), self.add_customer)


def run_simulation(arrival_rate, service_rate, run_until):

    schedule = Schedule()
    bus_system = BusSystem(arrival_rate, service_rate)
    bus_system.run(schedule)

    while schedule.next_event_time() < run_until:
        schedule.run_next_event()

    return bus_system

In [ ]:
bus_system = run_simulation(arrival_rate=1.2, service_rate=1, run_until=25)
print(f'There are {bus_system.queue.people_in_queue} people in the queue')

In [ ]:
class Simulation:
    '''
        Encapsulates a simulation run: instantiates the schedule and entities with starting values, 
        and runs as long as an event queue exists.
    '''
    def __init__(self):
        self.schedule  = Schedule()
        self.event_queue = []
        self.arrival_rate = 3

        #Setting distributions
        self.arrival_distribution  = lambda: sts.expon(scale = 1/self.arrival_rate)
        #Positive support as times can't be negative
        self.transit_time = lambda: sts.truncnorm.rvs(a = 0, b = 10, loc = 2, scale = 0.5)

    def setup(self):
        self.stops = [BusStop(i for i in range(10))]

    def run(self):
        while self.queue: 
            # Process the earliest event
            curr_event = heapq.heappop(self.event_queue)
            time, entity = curr_event.timestamp, curr_event.entity

            #Depending on the kind of entity, we decide how to process the event
            if isinstance(entity, BusStop):
                #An Arrival event at a bus stop
                entity.passenger_arrival(time, source = obj.stop_index)
                #Push next arrival to heap
                heapq.push(self.event_queue, Event(time + self.arrival_distribution, obj))
            elif isinstance(entity, Bus):
                # arrival at a bus stop
                if self.stops[entity.next_stop].stop_index == entity.next_stop:
                    curr_stop = self.stops[entity.next_stop]

                    #Customers exit
                    time_to_exit = entity.unload_passengers(curr_stop)
                    time_to_enter = curr_stop.board_bust(entity)
                    entity.next_stop += 1

                    heapq.heappush(time + time_to_enter + time_to_exit, entity)
                else:
                    #move to next stop
                    #Schedule next stop arrival
                    next_arrival = Event(time + self.transit_time, entity)
                    heapq.heappush(self.event_queue, next_arrival)
        

So, we have a stream of arrival times, sampled from an exponential distribution. This isn't a simulation. 

What we need is a rule to represent concurrency between the two sets of events. For starters, let's take a simple premise:

    You cannot arrive later than now. 
    if we are at t = 10 minutes, then you cannot arrive at t = 12.4 minutes.
    If we are at t = 12.4 minutes, you cannot arrive at t = 13.04 minutes.
        The number in the queue are those that arrived prior to now. 

But how do we increment our sense of now?
    There are two kinds of times that we register: arrival times, and service times. 
    Say we start at t = 0, and the first customer arrives at t = 1.05 min. How do we decide how to increment? 
    Easy, we service the first customer and set the time to 1.05 min. 
        But now you could say: what if a customer arrives at 0.07 min? 
        We sample inter-arrival times from an exponential distribution because they model *inter* arrival times. 
        In other words, once we take one measurement, we don't need to worry about previous ones. We can just resampling.
        But when do we take the next sample?
            The question is poking the anxiety that if you just keep stepping to the next arrival, then you never really build a queue. 
            Or develop a neat sense of time. 
        So what happens "now"? 
            Either we are currently servicing a customer, or we are "about" to service one. 
                Time here is discrete, but surreptitious: it evolves as events are processed. 
                It's almost like we are taking a series of discrete samples, and then labeling them as "arrivals" or "services".
                

A philosophy of 
 + Scheduling events
 + Handling Events
 + Events are atomic ie they have no duration. 


log_queue_length()

on Event 
    if arrival
        add to queue
        if queue length == 1
            scheduleEvent(now + B, )

# Make the queues rotate around the circuit


Entities
Gats
Queues 
Servers


Variables to track
Latency
Utilization 
Bottlenecks
            
# Bus Stop
1. Arrivals of passengers 

Bus
    1. Bus arrives at a stop
    2. Passengers at stop disembark with time N(x_t,sigma)
    3. Within capacity,

Limitations 
+ Buffers at each stop 

Reflection:


In [ ]:
class Event:
    '''
    Events are entities with timestamps. Processing logic is abstracted
    out to Simulation class. 
    '''

    def __init__(self, timestamp, entity) -> None:
        self.timestamp = timestamp
        self.entity  = entity

    def __lt__(self, other):
        return self.timestamp < other.timestamp


class Schedule:
    '''
    Implement an event schedule using a priority queue.
    Events are timestamps and objects : its easier to trace the execution logic
    At the simulation level
    '''
    
    def __init__(self):
        self.now = 0
        self.queue = []

    def add_event_at(self, timestamp, entity):
        heapq.heappush(self.queue, Event(timestamp, entity))

    def add_event_after(self, interval, entity):
        heapq.heappush(
            self.queue, 
            Event(self.now + interval, entity)
        )

    def get_next_event(self):
        next_event = heapq.heappop(self.queue)
        return next_event.timestamp, next_event.entity

    def next_event_time(self):
        # Min heaped so first entry is minimum of timestamp = earliest
        return self.queue[0].timestamp

    def print_events(self):
        for event in sorted(self.queue):
            print(f'   {event.timestamp}: {event.entity}')


In [ ]:
# To unit test our schedule, an abstraction over heapq, and events
# we create a dummy entity
# and a dummy simulation
import random


class BusStop:
    def __init__(self, stop_index) -> None:
        # customers waiting for bus
        self.stop_queue = []
        self.stop_index  = stop_index
        self.arrival_distribution = lambda: sts.expon.rvs(scale = 1)

    def customer_arrival(self, schedule, curr_time, n_stops = 5):

        # First come first served : either heap or sort before entering
        heapq.heappush(self.stop_queue, Passenger(arrival_time = curr_time, source = self.stop_index, total_stops = n_stops))

        #schedule next customer arrival
        schedule.add_event_after(self.arrival_distribution(), self)

        return len(self.stop_queue)


class Passenger:
    def __init__(self, arrival_time, source, total_stops = 5) -> None:
        self.arrival_time = arrival_time
        self.source_stop = source
        # Loop around, while mainting 1-start indexing for stops
        self.destination_stop = (self.source_stop + random.randint(0,6))%total_stops + 1

        self.departure_time = None

    def __lt__(self, other):
        return self.arrival_time < other.arrival_time


    def print_attrs(self):
        print(f'source is {self.source_stop} dest is {self.destination_stop}')


In [ ]:
schedule = Schedule()

schedule.add_event_at(2, BusStop(3))


while schedule.now < 25: 
    # take the next event
    curr_time, entity = schedule.get_next_event()

    if isinstance(entity, BusStop):
        # Let a customer arrive
        entity.customer_arrival(schedule, curr_time)
        #update time
        schedule.now = curr_time

In [ ]:
waiting_passengers  = schedule.queue[0].entity.stop_queue

waiting_passengers = [passenger.arrival_time for passenger in waiting_passengers]

waiting_passengers.sort()

plt.plot(waiting_passengers, range(1, len(waiting_passengers) + 1) )

Now we need to see if separate BusStop instances can grow their queues through repeatedly scheduling on the main queue. We are trying to understand robustness now.

We are also trying to figure out how to initialize the first event. 


In [ ]:
schedule  = Schedule()

#Initialize a circuit of 5 bus stops.

bus_stops = [BusStop(i + 1) for i in range(5)]
#load each onto the schedule as events
for stop in bus_stops:
    schedule.add_event_at(0, stop)

while schedule.now < 25: 
    # take the next event
    curr_time, entity = schedule.get_next_event()

    if isinstance(entity, BusStop):
        # Let a customer arrive
        entity.customer_arrival(schedule, curr_time)
        #update time
        schedule.now = curr_time


In [ ]:
#plot queue growth for each stop
for stop in bus_stops:
    waiting_passengers = [passenger.arrival_time for passenger in stop.stop_queue]
    waiting_passengers.sort()

    plt.plot(waiting_passengers, range(1, len(waiting_passengers) + 1))

plt.plot(range(1, len(waiting_passengers) + 1), range(1, len(waiting_passengers) + 1), color = 'black')

So our event schedule works. Now we need to implement a Bus entity, and develop methods that capture how the BusStop and Bus interact ie pass data to one another. 

Additionally, we want to be able to check how long a customer has been waiting: this is the duration betweeen when a customer arrives, and when they board the bus. 

In [ ]:
for stop in bus_stops:
    print([(passenger.source_stop, passenger.destination_stop) for passenger in stop.stop_queue])

Create a bus class, and make it move along the indices

In [ ]:
class Bus:
    def __init__(self, location, last_stop) -> None:
        # Index of the stop the bus is currently at
        self.location = location

        self._capacity = 130
        self._transit_time  = lambda: sts.truncnorm.rvs(0,10, loc = 2, scale = 0.5)
        self._last_stop = last_stop

        # Passengers riding the bus
        self.passengers  = []

    def move_to_next_stop(self, schedule, curr_time):
        #Update position
        if self.location < self._last_stop:
            self.location += 1
        else: 
            self.location = 1

        # Schedule next stop
        next_stop = self._transit_time()
        # Dummy sanity check
        self.passengers.append(f"At stop {self.location} at time {curr_time}. Next stop at time {curr_time + next_stop}")
        schedule.add_event_after(next_stop, self)
        

In [ ]:
schedule  = Schedule()

#Initialize a circuit of 5 bus stops.

# Instantiate Entities with sensible starting configs

total_stops = 5


bus_stops = [BusStop(i) for i in range(total_stops)]
#load each onto the schedule as events
for stop in bus_stops:
    schedule.add_event_at(0, stop)

# Create a bus at starting index and make it run in a loop
bus = Bus(0, total_stops)
schedule.add_event_at(0, bus)


while schedule.now < 25: 
    # take the next event
    curr_time, entity = schedule.get_next_event()

    if isinstance(entity, BusStop):
        # Let a customer arrive
        entity.customer_arrival(schedule, curr_time)
        #update time
        schedule.now = curr_time

    elif isinstance(entity, Bus):
        # Move Bus to next stop
        entity.move_to_next_stop(schedule, curr_time)
        schedule.now = curr_time

        


In [ ]:
bus.passengers

Now we augment the Bus object: we check if it is at a given BusStop, and pass the BusStop passengers into the bus within the Bus' capacity. Our check here is to see if it runs until its full, and Passengers are correctly transferred. 

Here we want to see if schedule.add_event_after(next_stop, self) passes the self at the time of the call to this function: if we add passengers after this call in the same self, will the same data update?

We also want synchronous updating - the bus travels after the boarding and disembarking happens

We also add the served_passengers variable to track all Passenger objects that disembarked the system ; for mean waiting times.

We will also do some preliminary plotting on the wait times, and see if they are sensible. Such experiments will be abstracted into a Data object later on. 

In [ ]:
class Bus:
    def __init__(self, location, last_stop, capacity = 130) -> None:
        # Index of the stop the bus is currently at
        self.location = location

        self._capacity = capacity
        self._transit_time  = lambda: sts.truncnorm.rvs(0,10, loc = 2, scale = 0.5)
        self._boarding_time = lambda n: sts.truncnorm.rvs(0,10, loc = 0.05*n, scale = 0.01*(n**0.5))
        self._last_stop = last_stop

        # Passengers riding the bus
        self.passengers  = []

    def move_to_next_stop(self, schedule, curr_time):
        #Update position in circuit
        if self.location < self._last_stop:
            self.location += 1
        else: 
            self.location = 1

        # Schedule next stop
        next_stop = self._transit_time()        
        schedule.add_event_after(next_stop, self)

    def load_passengers(self, schedule, curr_time, bus_stop):

        max_to_board  = self._capacity - len(self.passengers)
        n_to_board = min(len(bus_stop.stop_queue), max_to_board)

        time_to_load = self._boarding_time(n_to_board)

        print(f"Stop:{bus_stop.stop_index}, Passengers To Board:{n_to_board}, Remaining in Stop Queue: {len(bus_stop.stop_queue)}")


        for p in range(n_to_board):
            # exit the stop
            boarding_passenger = heapq.heappop(bus_stop.stop_queue)
            # Update the Passenger 
            boarding_passenger.departure_time = curr_time

            #print(f'Passenger boarding, {boarding_passenger.arrival_time} - {boarding_passenger.departure_time}')
            #add to passengers
            self.passengers.append(boarding_passenger)

        print(f"Stop:{bus_stop.stop_index}, Bus Pos: {self.location} Passengers on Bus:{len(self.passengers)} Remaining in Stop Queue: {len(bus_stop.stop_queue)}")

        
        return time_to_load

        

    

In [ ]:
schedule  = Schedule()

#Initialize a circuit of 5 bus stops.

# Instantiate Entities with sensible starting configs
total_stops = 5

served_passengers  = []

bus_stops = [BusStop(i + 1) for i in range(total_stops)]
#load each onto the schedule as events
for stop in bus_stops:
    schedule.add_event_at(0, stop)

# Create a bus at starting index and make it run in a loop
bus = Bus(location = 1, last_stop = total_stops, capacity = 10000)
schedule.add_event_at(0, bus)


while schedule.now < 50: 
    # take the next event
    curr_time, entity = schedule.get_next_event()

    if isinstance(entity, BusStop):
        # Let a customer arrive
        entity.customer_arrival(schedule, curr_time)
        #update time
        schedule.now = curr_time

    elif isinstance(entity, Bus):
        # check where bus is
        current_stop = bus_stops[entity.location - 1]

        #out_time = entity.unload_passengers(schedule, curr_time, current_stop)
        disembark_time = 0

        loading_time = entity.load_passengers(schedule, curr_time, current_stop)

        # Move Bus to next stop, after loading passenger and unloading passengers is complete
        entity.move_to_next_stop(schedule, curr_time + loading_time + disembark_time)
        
        schedule.now = curr_time


In [ ]:
wait_times = [(p.departure_time - p.arrival_time) for p in bus.passengers]
plt.hist(wait_times, bins = 20)

Now we update the Bus class to handle unloading. 


In [ ]:
# Now we handle unloading, and start collecting data more intentionally. 
# Inspecting classes at the end of a simulation run is painful.


class Bus:
    def __init__(self, location, last_stop, capacity = 130) -> None:
        # Index of the stop the bus is currently at
        self.location = location

        self._name = location
        self._capacity = capacity
        self._transit_time  = lambda: sts.truncnorm.rvs(0,10, loc = 2, scale = 0.5)
        self._boarding_time = lambda n: sts.truncnorm.rvs(0,10, loc = 0.05*n, scale = 0.01*(n**0.5))
        self._disembark_time = lambda n: sts.truncnorm.rvs(0,10, loc = 0.03*n, scale = 0.01*(n**0.5) )
        self._last_stop = last_stop

        # Passengers riding the bus
        self.passengers  = []

    def move_to_next_stop(self, schedule, curr_time):
        #Update position in circuit
        if self.location < self._last_stop:
            self.location += 1
        else: 
            self.location = 1

        # Schedule next stop
        next_stop = self._transit_time()        
        schedule.add_event_after(next_stop + curr_time, self)

    def load_passengers(self, curr_time, bus_stop):

        print(self._capacity, len(self.passengers))

        max_to_board  = self._capacity - len(self.passengers)
        n_to_board = min(len(bus_stop.stop_queue), max_to_board)

        time_to_load = self._boarding_time(n_to_board)

        #print("time_to_load", time_to_load, "stop", len(bus_stop.stop_queue), "max_board", max_to_board)

        #print(f"Stop:{bus_stop.stop_index}, Passengers To Board:{n_to_board}, Remaining in Queue: {len(bus_stop.stop_queue)}")

        # First come first serve
        for p in range(n_to_board):
            # exit the stop
            boarding_passenger = heapq.heappop(bus_stop.stop_queue)
            # Update the Passenger 
            boarding_passenger.departure_time = curr_time

            #print(f'Passenger boarding, {boarding_passenger.arrival_time} - {boarding_passenger.departure_time}')
            #add to passengers
            self.passengers.append(boarding_passenger)

        #print(f"Stop:{bus_stop.stop_index}, Bus Pos: {self.location} Passengers on Bus:{len(self.passengers)} Remaining in Queue: {len(bus_stop.stop_queue)}")

        return time_to_load

    def unload_passengers(self, bus_stop):
        #filter passengers who need to get off
        departing_passengers = list(filter(lambda p: p.destination_stop == bus_stop.stop_index, self.passengers))
        unload_time = self._disembark_time(len(departing_passengers))

        if len(self.passengers) > 90:
            #print(f'In Bus {self._name} at Location{self.location} : {len(self.passengers)}, to remove: {len(departing_passengers)}')

            for passenger in departing_passengers:
                self.passengers.remove(passenger)

            #print(f'In Bus {self._name} at Location{self.location} : {len(self.passengers)}, after removal')

        else: 
            for passenger in departing_passengers:
                self.passengers.remove(passenger)

        # Return total time, and the passengers exiting so we can collect data
        return unload_time, departing_passengers

In [ ]:
schedule  = Schedule()

#Initialize a circuit of 5 bus stops.
# TO DO: convert current time to HH:mm and limit at 24 hours. 

# Instantiate Entities with sensible starting configs
total_stops = 2

served_passengers  = []

bus_stops = [BusStop(i + 1) for i in range(total_stops)]
#load each onto the schedule as events
for stop in bus_stops:
    schedule.add_event_at(0, stop)

# Create a bus at starting index and make it run in a loop
bus = Bus(location = 1, last_stop = total_stops, capacity = 120)
schedule.add_event_at(0, bus)

times = []

while schedule.now < 1000: 
    # take the next event
    curr_time, entity = schedule.get_next_event()
    schedule.now = curr_time
    

    if isinstance(entity, BusStop):
        # Let a customer arrive

        entity.customer_arrival(schedule, curr_time)
        #update time
        

    elif isinstance(entity, Bus):
        # check where bus is
        current_stop = bus_stops[entity.location - 1]

        times.append((schedule.now, entity._name))

        #out_time = entity.unload_passengers(schedule, curr_time, current_stop)
        disembark_time, departed_customers = entity.unload_passengers(current_stop)
        served_passengers.append((curr_time, departed_customers, entity._name, entity.location))


        loading_time = entity.load_passengers(curr_time, current_stop)

        # Move Bus to next stop, after loading passenger and unloading passengers is complete
        entity.move_to_next_stop(schedule, loading_time + disembark_time)


In [ ]:
times

In [ ]:
plt.hist(np.diff(times), alpha = 0.3)
plt.hist(sts.expon.rvs(scale = 1, size = 1000), alpha = 0.5)

In [ ]:
#wait_times = [(p.departure_time - p.arrival_time) for p in served_passengers]

#plt.hist(wait_times)



data = list(zip(*served_passengers))



data[1] = [len(data[1][i]) for i in range(len(data[1]))]
print(data)


#scatter bc: are the departures happening every 2ish minutes? is there overlap in points?

#Check departures
plt.scatter(data[0], data[1], label = "departures at stop", c=data[2])

#check cumulative departures
data[1] = np.cumsum(data[1])
plt.scatter(data[0], data[1], label = "cumulative departures")
plt.legend()

So, times roughly end at 50, which was what we set, and the number of departures tend to increase but stay roughly random over time. Now we need to extend this to multiple buses, and handle for collisions, then collect more data. 

In [ ]:
schedule  = Schedule()

#Initialize a circuit of 5 bus stops.
# TO DO: convert current time to HH:mm and limit at 24 hours. 

# Instantiate Entities with sensible starting configs
total_stops = 5

served_passengers  = []

bus_stops = [BusStop(i + 1) for i in range(total_stops)]
#load each onto the schedule as events
for stop in bus_stops:
    schedule.add_event_at(0, stop)

# Create a bus at each starting index
buses = [Bus(location = i + 1, last_stop = total_stops, capacity = 10000) for i in range(5)]
for bus in buses:
    # TODO: check starting logic again
    schedule.add_event_at(0, bus)


while schedule.now < 24*60: 
    # take the next event
    curr_time, entity = schedule.get_next_event()

    if isinstance(entity, BusStop):
        # Let a customer arrive
        entity.customer_arrival(schedule, curr_time)
        #update time
        schedule.now = curr_time

    elif isinstance(entity, Bus):
        # check where bus is
        current_stop = bus_stops[entity.location - 1]

        #out_time = entity.unload_passengers(schedule, curr_time, current_stop)
        disembark_time, departed_customers = entity.unload_passengers(current_stop)
        served_passengers.append((curr_time, departed_customers, entity._name, entity.location))

        loading_time = entity.load_passengers(curr_time, current_stop)

        # Move Bus to next stop, after loading passenger and unloading passengers is complete
        entity.move_to_next_stop(schedule, curr_time + loading_time + disembark_time)
        schedule.now = curr_time


In [ ]:
# time, dept customers, Busname, location of departure

for bus in range(1, 6):
    bus_data = list(filter(lambda d: d[2] == bus, served_passengers))
    plot_data = list(zip(*bus_data))

    plot_data[1] = [len(plot_data[1][i]) for i in range(len(plot_data[1]))]

    #plt.scatter(plot_data[0], plot_data[1],c = plot_data[3])
    #plt.plot(plot_data[0], plot_data[1], label = f'Bus {bus} departures')

    plot_data[1] = np.cumsum(plot_data[1])

    plt.plot(plot_data[0], plot_data[1], label = f'Bus {bus}')

    plt.legend()

        # TO DO: convert current time to HH:mm and limit at 24 hours. 


In [ ]:
class Stats:
    def __init__(self, interval = 10) -> None:
        self.n_passengers = []
        self.interval = interval 

    def sample(self, schedule, curr_time, simulation):
        
        self.n_passengers.append((curr_time, [len(i.passengers) for i in simulation.buses]))

        schedule.add_event_after(self.interval, self)

In [ ]:
# Now lets create a simulation object to bring everything together
class Simulation:
    def __init__(self, total_buses = 5, final_time = 24*60) -> None:
        # params
        self.total_buses = total_buses
        self.total_stops = 15
        self.final_time  = final_time 

        # SIM variables:
        self.served_passengers  = []
        self.max_queue_length = 0
        


        # INIT sim
        self.schedule  = Schedule()

        self.bus_stops = [BusStop(i + 1) for i in range(self.total_stops)]
        #load each onto the schedule as events
        for stop in self.bus_stops:
            self.schedule.add_event_at(0, stop)

        # Create a bus at each starting index. Bus_n <= BusStop_n
        self.buses = [Bus(location = i + 1, last_stop = self.total_stops, capacity = 130) for i in range(self.total_buses)]
        for bus in self.buses:
            self.schedule.add_event_at(0, bus)

        self.stats = Stats()

        self.schedule.add_event_at(0, self.stats)


    def run(self):
        while self.schedule.now < self.final_time: 
            # take the next event
            curr_time, entity = self.schedule.get_next_event()
            self.schedule.now = curr_time


            if isinstance(entity, BusStop):
                # Let a customer arrive
                queue_length = entity.customer_arrival(self.schedule, curr_time, self.total_stops)
                
                if self.max_queue_length < queue_length:
                    self.max_queue_length = queue_length

            elif isinstance(entity, Bus):
                # check where bus is
                current_stop = self.bus_stops[entity.location - 1]

                disembark_time, departed_customers = entity.unload_passengers(current_stop)
                self.served_passengers.append((curr_time, departed_customers, entity._name, entity.location))

                loading_time = entity.load_passengers(curr_time, current_stop)

                # Move Bus to next stop, after loading passenger and unloading passengers is complete
                entity.move_to_next_stop(self.schedule, curr_time + loading_time + disembark_time)

            elif isinstance(entity, Stats):
                entity.sample(self.schedule, curr_time, self)

    def plot(self):
        # a kind of sanity check plot
        # a good plot doesn't top off (bus unloading/loading works well)

        for bus in range(1, self.total_buses + 1):
            bus_data = list(filter(lambda d: d[2] == bus, self.served_passengers))
            plot_data = list(zip(*bus_data))

            plot_data[1] = [len(plot_data[1][i]) for i in range(len(plot_data[1]))]

            #plt.scatter(plot_data[0], plot_data[1],c = plot_data[3])
            #plt.plot(plot_data[0], plot_data[1], label = f'Bus {bus} departures')

            plot_data[1] = np.cumsum(plot_data[1])

            plt.plot(plot_data[0], plot_data[1], label = f'Bus {bus} Cumulative Departures')

            plt.legend()

    def wait_times(self):
        data = list(zip(*self.served_passengers))

        # compute wait times
        wait_times = [[p.departure_time - p.arrival_time for p in el] for el in data[1]]
        # flatten array
        wait_times = reduce(lambda x,y: x + y, wait_times)

        expected_val = np.mean(wait_times)
        
        plt.hist(wait_times, bins = 100)
        plt.axvline(expected_val, color = "red", linewidth = 1, linestyle = '--', label = f"Mean waiting time = {round(expected_val)} mins")
        plt.ylabel("Frequency")
        plt.xlabel("Passenger Waiting Time (minutes)")
        plt.title(f"Passenger Waiting Times for {self.total_buses} buses")
        plt.legend()

        return expected_val
    




In [ ]:
sim = Simulation(total_buses=1)

sim.run()
sim.plot()

# Bus capacity over each hour, for every bus
# Count losses


In [ ]:
def transpose_to_plot(data):
    return list(zip(*data))


def mins_to_hours(times):
    t = np.array(times)

    return t//60 


plot_data = transpose_to_plot(sim.stats.n_passengers)

t = plot_data[0]
y = [np.mean(plot_data[1][i]) for i in range(len(plot_data[1]))]

plt.plot(t, y)


In [ ]:
data = list(zip(*sim.served_passengers))

def inverse_mod(x):
    if x < 0:
        return 15 + x
    else:
        return x

# compute stop
stops = [[p.destination_stop - p.source_stop for p in el] for el in data[1]]
# flatten array
stops = reduce(lambda x,y: x + y, stops)
# handle wrap around
stops = list(map(inverse_mod, stops))


plt.hist(stops, bins = 7)
plt.xlabel("Number of stops")
plt.ylabel("Frequency")

In [ ]:
mean = sim.wait_times()

In [ ]:
class BatchRunner:
    def __init__(self, total_buses = 5, run_count = 1) -> None:
        self.total_buses = total_buses
        self.run_count = run_count
        self.sim_data  = None
        self.plot_data = lambda: list(zip(*self.sim_data.items()))

    def batch_run(self):
        buses = range(1, self.total_buses + 1)
        sim_data = {bus: {"avg_wait": 0, "max_queue":0} for bus in buses}

        for n_bus in buses:

            mean_wait_times = []
            max_queues = []

            for sim_run in range(self.run_count):
                this_sim = Simulation(total_buses=n_bus)

                this_sim.run()

                mean = this_sim.wait_times()
                mean_wait_times.append(mean)

                max_queue = this_sim.max_queue_length
                max_queues.append(max_queue)

            print(mean_wait_times)
            m = np.mean(mean_wait_times)
            t = sts.sem(mean_wait_times)
            sim_data[n_bus]["avg_wait"] = (m, 1.96*t)

            m_max = np.mean(max_queues)
            t_max = sts.sem(max_queues)

            sim_data[n_bus]["max_queue"] = (m_max, 1.96*t_max)
        
        self.sim_data = sim_data

        return self.sim_data

    def wait_times_plot(self):
        plot_data = self.plot_data()

        x = plot_data[0]

        y = [i["avg_wait"][0] for i in plot_data[1]]
        err = [i["avg_wait"][1] for i in plot_data[1]]

        plt.plot(x,y, color = "gray")
        plt.errorbar(x,y, err, fmt= '.k')
        plt.xlabel("Number of Buses")
        plt.ylabel("Mean Waiting Time (minutes)")
        plt.show()


    def max_queue_plot(self):
        plot_data = self.plot_data()

        x = plot_data[0]

        y = [i["max_queue"][0] for i in plot_data[1]]
        err = [i["max_queue"][1] for i in plot_data[1]]

        plt.plot(x,y, color = "gray")
        plt.errorbar(x,y, err, fmt= '.k')
        plt.xlabel("Number of Buses")
        plt.ylabel("Max Queue Length (number of waiting passengers)")


In [ ]:
batch = BatchRunner()

data = batch.batch_run()

In [ ]:
plot_data

In [ ]:
x = plot_data[0]

y = [i["max_queue"][0] for i in plot_data[1]]
err = [i["max_queue"][1] for i in plot_data[1]]

plt.plot(x,y, color = "gray")
plt.errorbar(x,y, err, fmt= '.k')
plt.xlabel("Number of Buses")
plt.ylabel("Max Queue Length (number of waiting passengers)")


In [ ]:
# another quantity: mean number of passengers in bus

# mean passengers in buses, over the day
# at t = 30, [10, 20, 40, 30] => mean passengers is 20

In [ ]:
# lambda = 1.2 + cos(pi*(t-7)/6)
# t in hours --> convert to hours 

# track passengers who wait too long

# faster to take the bus or walk? 


# event queue visualization
# Bus
# BusStop

In [ ]:
25**0.5